In [1]:
import sempy.fabric as fabric

result=fabric.list_workspaces().query("Name=='Sales API'")
workspaceId=result.Id.to_list()[0]

itemResult=fabric.list_items(workspace=workspaceId).query("Type=='Lakehouse'")

itemId=itemResult.Id.to_list()[0]

StatementMeta(, 79848811-9788-41f9-a37b-f66fd3ac5343, 3, Finished, Available, Finished)

In [2]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
import pandas as pd
from pyspark.sql.functions import col, lit
from functools import reduce


client=fabric.FabricRestClient()


url=f"v1/workspaces/{workspaceId}/items/{itemId}/shortcuts"
result=client.get(url)

df_items = pd.json_normalize(result.json()['value'])

df=spark.createDataFrame(df_items)

df=df.select("name","path",col("`target.type`").alias("type"),col("`target.oneLake.workspaceId`").alias("workspaceId"),col("`target.oneLake.itemId`").alias("itemId"),col("`target.oneLake.path`").alias("objectPath"))

display(df)

StatementMeta(, 79848811-9788-41f9-a37b-f66fd3ac5343, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fc2eed39-6467-4827-a432-40d334f717fe)

In [3]:
dfWorkspaces=df.select(col("workspaceId").cast("string"))

dfWorkspaces=dfWorkspaces.dropDuplicates()

existingWkList=[wk["workspaceId"]  for wk in dfWorkspaces.collect()]

workspaceList=spark.createDataFrame(fabric.list_workspaces())

workspaces=workspaceList.join(dfWorkspaces,workspaceList.Id==dfWorkspaces.workspaceId)

workspaces=workspaces.select("workspaceId","Name")

shortcuts=df.alias("df").join(workspaces.alias("workspaces"),"workspaceId").select("df.*",col("workspaces.Name").alias("workspaceName"))

display(shortcuts)

StatementMeta(, 79848811-9788-41f9-a37b-f66fd3ac5343, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3011ae34-7791-4a2f-950d-80d210280493)

In [4]:
def processItemList(wk):
    idf=spark.createDataFrame(fabric.list_items(workspace=wk))
    idf=idf.withColumn("workspaceId", lit(wk))
    return idf

StatementMeta(, 79848811-9788-41f9-a37b-f66fd3ac5343, 6, Finished, Available, Finished)

In [5]:
wkList=spark.createDataFrame(fabric.list_workspaces())

condition=wkList["Id"].cast("string").isin(existingWkList)

wkList=wkList.alias("wkList").filter(condition)

df_list=[processItemList(ws["Id"]) for ws in wkList.collect()]

union_df = reduce(lambda df1, df2: df1.union(df2), df_list)

joinCondition= (shortcuts.workspaceId==union_df.workspaceId) & (shortcuts.itemId==union_df.Id)

shortcuts=shortcuts.alias("shortcuts").join(union_df.alias("items"),joinCondition)

shortcuts=shortcuts.select("shortcuts.*",union_df["Display Name"].alias("ItemName"),union_df["Type"].alias("itemType"))

display(shortcuts)

StatementMeta(, 79848811-9788-41f9-a37b-f66fd3ac5343, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 01fe85f2-b7d7-447f-8d9c-0082d9bc5f7a)